---

_You are currently looking at **version 1.1** of this notebook. To download notebooks and datafiles, as well as get help on Jupyter notebooks in the Coursera platform, visit the [Jupyter Notebook FAQ](https://www.coursera.org/learn/python-data-analysis/resources/0dhYG) course resource._

---

In [29]:
import pandas as pd
import numpy as np
from scipy.stats import ttest_ind

# Assignment 4 - Hypothesis Testing
This assignment requires more individual learning than previous assignments - you are encouraged to check out the [pandas documentation](http://pandas.pydata.org/pandas-docs/stable/) to find functions or methods you might not have used yet, or ask questions on [Stack Overflow](http://stackoverflow.com/) and tag them as pandas and python related. And of course, the discussion forums are open for interaction with your peers and the course staff.

Definitions:
* A _quarter_ is a specific three month period, Q1 is January through March, Q2 is April through June, Q3 is July through September, Q4 is October through December.
* A _recession_ is defined as starting with two consecutive quarters of GDP decline, and ending with two consecutive quarters of GDP growth.
* A _recession bottom_ is the quarter within a recession which had the lowest GDP.
* A _university town_ is a city which has a high percentage of university students compared to the total population of the city.

**Hypothesis**: University towns have their mean housing prices less effected by recessions. Run a t-test to compare the ratio of the mean price of houses in university towns the quarter before the recession starts compared to the recession bottom. (`price_ratio=quarter_before_recession/recession_bottom`)

The following data files are available for this assignment:
* From the [Zillow research data site](http://www.zillow.com/research/data/) there is housing data for the United States. In particular the datafile for [all homes at a city level](http://files.zillowstatic.com/research/public/City/City_Zhvi_AllHomes.csv), ```City_Zhvi_AllHomes.csv```, has median home sale prices at a fine grained level.
* From the Wikipedia page on college towns is a list of [university towns in the United States](https://en.wikipedia.org/wiki/List_of_college_towns#College_towns_in_the_United_States) which has been copy and pasted into the file ```university_towns.txt```.
* From Bureau of Economic Analysis, US Department of Commerce, the [GDP over time](http://www.bea.gov/national/index.htm#gdp) of the United States in current dollars (use the chained value in 2009 dollars), in quarterly intervals, in the file ```gdplev.xls```. For this assignment, only look at GDP data from the first quarter of 2000 onward.

Each function in this assignment below is worth 10%, with the exception of ```run_ttest()```, which is worth 50%.

In [30]:
# Use this dictionary to map state names to two letter acronyms
states = {'OH': 'Ohio', 'KY': 'Kentucky', 'AS': 'American Samoa', 'NV': 'Nevada', 'WY': 'Wyoming', 'NA': 'National', 'AL': 'Alabama', 'MD': 'Maryland', 'AK': 'Alaska', 'UT': 'Utah', 'OR': 'Oregon', 'MT': 'Montana', 'IL': 'Illinois', 'TN': 'Tennessee', 'DC': 'District of Columbia', 'VT': 'Vermont', 'ID': 'Idaho', 'AR': 'Arkansas', 'ME': 'Maine', 'WA': 'Washington', 'HI': 'Hawaii', 'WI': 'Wisconsin', 'MI': 'Michigan', 'IN': 'Indiana', 'NJ': 'New Jersey', 'AZ': 'Arizona', 'GU': 'Guam', 'MS': 'Mississippi', 'PR': 'Puerto Rico', 'NC': 'North Carolina', 'TX': 'Texas', 'SD': 'South Dakota', 'MP': 'Northern Mariana Islands', 'IA': 'Iowa', 'MO': 'Missouri', 'CT': 'Connecticut', 'WV': 'West Virginia', 'SC': 'South Carolina', 'LA': 'Louisiana', 'KS': 'Kansas', 'NY': 'New York', 'NE': 'Nebraska', 'OK': 'Oklahoma', 'FL': 'Florida', 'CA': 'California', 'CO': 'Colorado', 'PA': 'Pennsylvania', 'DE': 'Delaware', 'NM': 'New Mexico', 'RI': 'Rhode Island', 'MN': 'Minnesota', 'VI': 'Virgin Islands', 'NH': 'New Hampshire', 'MA': 'Massachusetts', 'GA': 'Georgia', 'ND': 'North Dakota', 'VA': 'Virginia'}

In [31]:
'''Returns a DataFrame of towns and the states they are in from the 
university_towns.txt list. The format of the DataFrame should be:
DataFrame( [ ["Michigan", "Ann Arbor"], ["Michigan", "Yipsilanti"] ], 
columns=["State", "RegionName"]  )
    
The following cleaning needs to be done:

1. For "State", removing characters from "[" to the end.
2. For "RegionName", when applicable, removing every character from " (" to the end.
3. Depending on how you read the data, you may need to remove newline character '\n'. '''

def get_list_of_university_towns():

    #reading data
    f = (open("university_towns.txt")).read()
    f = f.split('\n')[0:-1]
    
    #set up 2d list
    towns_list = []
    state = ''
    
    #iterate thru txt, adding towns in desired, cleaned format
    for i in f:
        if i[-6:] == '[edit]':
            state = i[:-6]
        else:
            towns_list.append([state, i.split(' (')[0]])
    
    #returning info in a df as desired
    return pd.DataFrame(towns_list, columns = ['State', 'RegionName'])

In [32]:
'''Returns the year and quarter of the recession start time as a 
string value in a format such as 2005q3'''

def get_recession_start():
    
    #reading and clearing data we're not addressing
    df = pd.read_excel('gdplev.xls', skiprows = 7,
                       usecols = ['Unnamed: 4', 'Unnamed: 6'])
    df.columns = ['Quarter', 'GDP in Billions (2009 Dollars)']
    
    #looking for the starting quarter, 2000q1
    first = df[df['Quarter']=='2000q1'].index.values
    df = df.iloc[int(first):]
    
    df = df.reset_index()
    del df['index']
    
    #looking for declining quarters
    df['Net Change'] = df['GDP in Billions (2009 Dollars)'].diff()
    df['Decline'] = (df['Net Change'] < 0)
    
    #only leaving declining quarters followed by another decline
    df['Start'] = (df['Decline'] & df['Decline'].shift(-1))
    df = df[df['Start']]
    
    #return the first instance
    return df.reset_index().loc[0, 'Quarter']

In [33]:
'''Returns the year and quarter of the recession end time as a 
    string value in a format such as 2005q3'''

def get_recession_end():
    
    #reading and clearing data we're not addressing
    df = pd.read_excel('gdplev.xls', skiprows = 7,
                       usecols = ['Unnamed: 4', 'Unnamed: 6'])
    df.columns = ['Quarter', 'GDP in Billions (2009 Dollars)']
    
    #looking for the recession start quarter
    first = df[df['Quarter']==get_recession_start()].index.values
    df = df.iloc[int(first):]
       
    df = df.reset_index()
    del df['index']
    
    #looking for growing quarters
    df['Net Change'] = df['GDP in Billions (2009 Dollars)'].diff()
    df['Growth'] = (df['Net Change'] > 0)
    
    #only leaving growth quarters preceded by another growth
    df['End'] = (df['Growth'] & df['Growth'].shift(1))
    df = df[df['End']]
    
    #return the first instance
    return df.reset_index().loc[0, 'Quarter']


In [34]:
'''Returns the year and quarter of the recession bottom time as a 
string value in a format such as 2005q3'''

def get_recession_bottom():
    
    #reading and clearing data we're not addressing
    df = pd.read_excel('gdplev.xls', skiprows = 7,
                       usecols = ['Unnamed: 4', 'Unnamed: 6'])
    df.columns = ['Quarter', 'GDP in Billions (2009 Dollars)']
    
    #looking for the recession
    first = df[df['Quarter']==get_recession_start()].index.values
    last = df[df['Quarter']==get_recession_end()].index.values
    df = df.iloc[int(first):int(last)]
    
    df = df.reset_index()
    del df['index']
    
    #looking for lowest gdp and where it is
    min_GDP = df['GDP in Billions (2009 Dollars)'].min()
    bottom_index = df[df['GDP in Billions (2009 Dollars)']==min_GDP].index.values
    
    #returning the quarter
    return df.loc[int(bottom_index), 'Quarter']

In [35]:
'''Converts the housing data to quarters and returns it as mean 
values in a dataframe. This dataframe should be a dataframe with
columns for 2000q1 through 2016q3, and should have a multi-index
in the shape of ["State","RegionName"].
    
Note: Quarters are defined in the assignment description, they are
not arbitrary three month periods.
    
The resulting dataframe should have 67 columns, and 10,730 rows.
'''


def convert_housing_data_to_quarters():
    
    states = {'OH': 'Ohio', 'KY': 'Kentucky', 'AS': 'American Samoa', 'NV': 'Nevada', 'WY': 'Wyoming', 'NA': 'National', 'AL': 'Alabama', 'MD': 'Maryland', 'AK': 'Alaska', 'UT': 'Utah', 'OR': 'Oregon', 'MT': 'Montana', 'IL': 'Illinois', 'TN': 'Tennessee', 'DC': 'District of Columbia', 'VT': 'Vermont', 'ID': 'Idaho', 'AR': 'Arkansas', 'ME': 'Maine', 'WA': 'Washington', 'HI': 'Hawaii', 'WI': 'Wisconsin', 'MI': 'Michigan', 'IN': 'Indiana', 'NJ': 'New Jersey', 'AZ': 'Arizona', 'GU': 'Guam', 'MS': 'Mississippi', 'PR': 'Puerto Rico', 'NC': 'North Carolina', 'TX': 'Texas', 'SD': 'South Dakota', 'MP': 'Northern Mariana Islands', 'IA': 'Iowa', 'MO': 'Missouri', 'CT': 'Connecticut', 'WV': 'West Virginia', 'SC': 'South Carolina', 'LA': 'Louisiana', 'KS': 'Kansas', 'NY': 'New York', 'NE': 'Nebraska', 'OK': 'Oklahoma', 'FL': 'Florida', 'CA': 'California', 'CO': 'Colorado', 'PA': 'Pennsylvania', 'DE': 'Delaware', 'NM': 'New Mexico', 'RI': 'Rhode Island', 'MN': 'Minnesota', 'VI': 'Virgin Islands', 'NH': 'New Hampshire', 'MA': 'Massachusetts', 'GA': 'Georgia', 'ND': 'North Dakota', 'VA': 'Virginia'}
    
    df = pd.read_csv('City_Zhvi_AllHomes.csv')
    df['State'].replace(states, inplace = True)
    
    df.set_index(["State","RegionName"], inplace = True)
    df.sort_index(inplace = True)
    
    for column in df.columns:
        if column[0:2] != '20':
            del df[column]
    
    year = '2000'
    quarter = 'q1'
    q_count = 1
    y_count = 1
    
    for column in df.columns:
        
        if (year + quarter) not in df.columns:
            df[year + quarter] = 0
        df[year + quarter] += df[column]
        
        if (q_count %3) == 0:
            df[year + quarter] = df[year + quarter]/3
            quarter = 'q' + str((int(quarter[1:])%4)+1)
            
        if (y_count % 12) == 0:
            year = str(int(year)+1)

        y_count += 1   
        q_count += 1
        
        del df[column]
        
    df[year + quarter] /= 2
    
    return df


In [41]:
'''First creates new data showing the decline or growth of housing prices
between the recession start and the recession bottom. Then runs a ttest
comparing the university town values to the non-university towns values, 
return whether the alternative hypothesis (that the two groups are the same)
is true or not as well as the p-value of the confidence. 
    
Return the tuple (different, p, better) where different=True if the t-test is
True at a p<0.01 (we reject the null hypothesis), or different=False if 
otherwise (we cannot reject the null hypothesis). The variable p should
be equal to the exact p value returned from scipy.stats.ttest_ind(). The
value for better should be either "university town" or "non-university town"
depending on which has a lower mean price ratio (which is equivilent to a
reduced market loss).'''

def run_ttest():
    
    #getting all the data in the form we want
    df = convert_housing_data_to_quarters()
    start = get_recession_start()
    bottom = get_recession_bottom()
    end = get_recession_end()
    
    cols_to_keep = [start, bottom]
    df = df[cols_to_keep]
    
    #calculating decline and ratio for towns
    df['delta'] = df[bottom] - df[start]
    df['Price Ratio'] = df[start]/df[bottom]
    
    #distinguishing uni towns from non-uni towns on housing df
    university_towns = get_list_of_university_towns()
    university_towns['University Towns'] = True
    university_towns.set_index(['State', 'RegionName'], inplace = True)
    
    df = pd.merge(df, university_towns, how = 'left', left_index = True, right_index = True)
    df['University Towns'].fillna(False, inplace = True)
    
    #grouping housing df by uni/non-uni, and getting rid of missing info 
    df = df[(np.isnan(df['delta'])==False)]
    uni_towns = df[df['University Towns']]
    n_uni_towns = df[df['University Towns']==False]
    
    #actually doing the test
    test_result = ttest_ind(uni_towns['delta'], n_uni_towns['delta'])
    
    #formatting result
    different = (test_result.pvalue < 0.01)
    better = 'non-university town'
    
    if uni_towns['Price Ratio'].mean() < n_uni_towns['Price Ratio'].mean():
        better = 'university town'
    
    return (different, test_result.pvalue, better)

Ttest_indResult(statistic=2.8324726966688369, pvalue=0.0046283242533887627)